It seems that coefficients decrease in a sterotyped fashion - understanding how is important to apply that to image compression but also to understand generative models for images.

In [8]:
from __future__ import division, print_function
import matplotlib
#matplotlib.rcParams.update({'font.size': 18, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
matplotlib.rcParams.update({'font.size': 18, 'text.usetex': True})
%matplotlib inline
%config InlineBackend.figure_format='svg'
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=2, suppress=True)
%cd test

/Users/lolo/pool/science/BICV-book/SI_BICV_sparse/SparseEdges/test


# fits decreases in RMSE

In [ ]:
import numpy as np
from SparseEdges import SparseEdges
import sys
mp = SparseEdges('https://raw.githubusercontent.com/meduz/SparseEdges/master/default_param.py')
mp.pe.datapath = '../../SLIP/database/'

threshold = None # classical plots
threshold = .5 # plot sparseness obtained when reaching this threshold

mp = SparseEdges('https://raw.githubusercontent.com/meduz/SparseEdges/master/default_param.py')
def init_mp():
    mp = SparseEdges('https://raw.githubusercontent.com/meduz/SparseEdges/master/default_param.py')
    mp.pe.seed = 42
    mp.pe.N_image = 60
    mp.pe.datapath = '../../SLIP/database/'
    return mp

import matplotlib.pyplot as plt
fig_width_pt = 318.67085 # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
fig_width = fig_width_pt*inches_per_pt  # width in inches

mps, experiments = [], []
v_B_sf = np.logspace(-.2, .2, 5, base=10, endpoint=True)*mp.pe.B_sf
for B_sf in v_B_sf:
    mp = init_mp()
    mp.pe.B_sf = B_sf
    exp = 'efficiency_B_sf_' + str(B_sf).replace('.', '_')
    mp.process(exp)
    experiments.append(exp)
    mps.append(mp)

databases = ['serre07_distractors'] * len(experiments)
labels = ['%0.2f' % B_sf for B_sf in v_B_sf]
try:
    fig, A, inset = mp.plot(mps=mps,
                      experiments=experiments, databases=databases, labels=labels, ref=2,
                      fig=fig, ax=A, color=[0., 1., 0.], threshold=threshold, scale=False)    
    A.set_xlabel(r'frequency bandwith $B_{sf}$')
    #A.set_yticks([0., 0.02, 0.04, 0.06])
except Exception as e:
    print('Failed to plot  with error : %s ' % e )



In [ ]:
fig, [A, B] = plt.subplots(1, 2, figsize=(13, 5), subplot_kw={'axisbg':'w'})
A.set_color_cycle(np.array([[1., 0., 0.]]))
for mp, experiment, name_database, label in zip(mps, experiments, databases, labels):
    try:
        imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
        RMSE /= RMSE[:, 0][:, np.newaxis]
        #print RMSE.shape, edgeslist.shape
        value = edgeslist[4, ...]
        #value /= value[0, :][np.newaxis, :]
        value /= RMSE[:, 0][np.newaxis, :]
        
        B.semilogx( value, alpha=.7)
        
        A.semilogx( RMSE.T, alpha=.7)
        A.set_xlabel('l0')
        A.axis('tight')
        A.axis('tight')
        _ = A.set_ylabel('RMSE')
    except Exception, e:
        print('Failed to plot experiment %s with error : %s ' % (experiment, e) )



In [ ]:
fig, A = plt.subplots(1, 1, figsize=(13, 13), subplot_kw={'axisbg':'w'})
from lmfit.models import ExpressionModel
#mod = PowerLawModel()
mod = ExpressionModel('1 - (1- eps_inf) * ( 1 - rho**x)')

threshold = .5

for mp, experiment, name_database, label in zip(mps, experiments, databases, labels):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    for i_image in range(RMSE.shape[0]):
        #pars = mod.guess(RMSE[i_image, :], x=np.arange(N))
        mod.def_vals = {'eps_inf':.1, 'rho':.99}
        out  = mod.fit(RMSE[i_image, :], x=np.arange(N))
        #print(out.fit_report(min_correl=0.25))
        eps_inf = out.params.get('eps_inf').value
        rho =  out.params.get('rho').value
        #print rho, eps_inf
        N_theta = np.log((threshold-eps_inf)/(1-eps_inf))/np.log(rho)
        #print N_theta
        A.semilogx( RMSE[i_image, :], alpha=.7)
        params = mod.make_params(eps_inf=eps_inf, rho=rho)
        A.semilogx(mod.eval(params, x=np.arange(N)), 'r--', alpha=.7)
        A.set_xlabel('l0')
        A.axis('tight')
        A.axis('tight')
        _ = A.set_ylabel('RMSE')            



In [ ]:
fig, A = plt.subplots(1, 1, figsize=(13, 13), subplot_kw={'axisbg':'w'})
from lmfit.models import ExpressionModel
#mod = PowerLawModel()
mod = ExpressionModel('rho**(1./(1.+(x/N50)**2)**.5)')

threshold = .5

for mp, experiment, name_database, label in zip(mps, experiments, databases, labels):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    for i_image in range(RMSE.shape[0]):
        #pars = mod.guess(RMSE[i_image, :], x=np.arange(N))
        mod.def_vals = {'N50':100., 'rho':.99}
        out  = mod.fit(RMSE[i_image, :], x=np.arange(N))
        #print(out.fit_report(min_correl=0.25))
        rho =  out.params.get('rho').value
        N50 =  out.params.get('N50').value
        print 'rho=', rho
        #N_theta = np.log(threshold)/np.log(rho)
        #print N_theta
        A.semilogx( RMSE[i_image, :], alpha=.7)
        params = mod.make_params(N50=N50, rho=rho)
        A.semilogx(mod.eval(params, x=np.arange(N)), 'r--', alpha=.7)
        A.set_xlabel('l0')
        A.axis('tight')
        A.axis('tight')
        _ = A.set_ylabel('RMSE')     
        



In [ ]:
fig, A = plt.subplots(1, 1, figsize=(13, 13), subplot_kw={'axisbg':'w'})
from lmfit.models import ExpressionModel
#mod = PowerLawModel()
mod = ExpressionModel('rho**x')

threshold = .5

for mp, experiment, name_database, label in zip(mps, experiments, databases, labels):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    for i_image in range(RMSE.shape[0]):
        #pars = mod.guess(RMSE[i_image, :], x=np.arange(N))
        mod.def_vals = {'rho':.99}
        out  = mod.fit(RMSE[i_image, :], x=np.arange(N))
        #print(out.fit_report(min_correl=0.25))
        rho =  out.params.get('rho').value
        print 'rho=', rho
        #N_theta = np.log(threshold)/np.log(rho)
        #print N_theta
        A.semilogx( RMSE[i_image, :], alpha=.7)
        params = mod.make_params(rho=rho)
        A.semilogx(mod.eval(params, x=np.arange(N)), 'r--', alpha=.7)
        A.set_xlabel('l0')
        A.axis('tight')
        A.axis('tight')
        _ = A.set_ylabel('RMSE')            



In [ ]:
fig, A = plt.subplots(1, 1, figsize=(13, 13), subplot_kw={'axisbg':'w'})

imagelist_ref, edgeslist_ref, RMSE_ref = mps[2].process(exp=experiments[2], name_database=name_database)
RMSE_ref /= RMSE_ref[:, 0][:, np.newaxis]

colors = [plt.cm.hsv(hsv) for hsv in np.linspace(0., 1., len(mps))]
for mp, experiment, name_database, label, color in zip(mps, experiments, databases, labels, colors):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    for i_image in range(RMSE.shape[0]):
        A.semilogx( RMSE[i_image, :]/RMSE_ref[i_image, :], alpha=.1, color=color)
        A.set_xlabel('l0')
        A.axis('tight')
        A.axis('tight')
        _ = A.set_ylabel('RMSE')            
    print (RMSE/RMSE_ref).mean(), (RMSE/RMSE_ref).std(axis=0).mean(), (RMSE/RMSE_ref).std(axis=0).shape


In [ ]:
mod = ExpressionModel('rho**x')

threshold = .5

def fit_rho(mp, experiment, name_database):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    rho = np.zeros(RMSE.shape[0])
    for i_image in range(RMSE.shape[0]):
        mod.def_vals = {'rho':.99}
        out  = mod.fit(RMSE[i_image, :], x=np.arange(N))
        rho[i_image] =  out.params.get('rho').value
    return rho

rho = fit_rho(mps[2], experiment, name_database=name_database)
L0_ref = np.log(threshold)/np.log(rho)
print L0_ref
for mp, experiment, name_database in zip(mps, experiments, databases):
    rho = fit_rho(mp, experiment, name_database=name_database)
    L0 = np.log(threshold)/np.log(rho)
    print L0
    print (L0/L0_ref).mean(), (L0/L0_ref).std(), L0.shape


In [ ]:

threshold = .5

def fit_L0(mp, experiment, name_database, threshold=threshold):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    L0 =  np.argmax(RMSE<threshold, axis=1)*1.
    return L0

L0_ref = fit_L0(mps[2], experiment, name_database=name_database)
print L0_ref
for mp, experiment, name_database in zip(mps, experiments, databases):
    L0 = fit_rho(mp, experiment, name_database, threshold)
    print L0
    print (L0/L0_ref).mean(), (L0/L0_ref).std(), L0.shape


In [ ]:
rect = plt.bar(np.arange(L0.shape[0]), L0)

In [ ]:
r = rect[0]

In [ ]:
r.set_color('w')
r.set_edgecolor('b')

In [ ]:
try:
    fig, A, ax = plot(mps=mps,
                      experiments=experiments, databases=databases, labels=labels, 
                      fig=fig, ax=A, color=[0., 1., 0.], threshold=threshold, scale=True)    
    A.set_xlabel(r'frequency bandwith $B_{sf}$')
    A.set_yticks([0., 0.02, 0.04, 0.06])
except Exception, e:
    print('Failed to plot  with error : %s ' % e )
    

In [ ]:
plt.cm.hsv(0.1)


## quantization

In [ ]:
import matplotlib
#matplotlib.rcParams.update({'font.size': 18, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
matplotlib.rcParams.update({'font.size': 18, 'text.usetex': True})
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=2, suppress=True)

#import __init__
import numpy as np
from NeuroTools.parameters import ParameterSet
from SLIP import Image
from LogGabor import LogGabor
from SparseEdges import SparseEdges, plot
import sys
pe = ParameterSet('default_param.py')
pe.seed = 42 # this ensures that all image lists are the same for the different experiments
pe.N_image = 9
pe.N = 512
im = Image(pe)
lg = LogGabor(im)
mp = SparseEdges(lg)
name_database = 'serre07_distractors'
imageslist, edgeslist, RMSE = mp.process(exp='prior_vanilla', name_database=name_database)
imageslist, edgeslist, RMSE_prior =  mp.process(exp='prior_firstorder', name_database=name_database)

In [ ]:
print RMSE.shape
plt.hist(RMSE[:, -1]/RMSE_prior[:, -1])

In [ ]:
fig, A = plt.subplots(1, 1, figsize=(13, 13), subplot_kw={'axisbg':'w'})
from lmfit.models import ExpressionModel
#mod = PowerLawModel()
mod = ExpressionModel('rho**x')

threshold = .5

for mp, experiment, name_database, label in zip(mps, experiments, databases, labels):
    imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
    RMSE /= RMSE[:, 0][:, np.newaxis]
    N = RMSE.shape[1] #number of edges
    for i_image in range(RMSE.shape[0]):
        #pars = mod.guess(RMSE[i_image, :], x=np.arange(N))
        mod.def_vals = {'rho':.99}
        out  = mod.fit(RMSE[i_image, :], x=np.arange(N))
        #print(out.fit_report(min_correl=0.25))
        rho =  out.params.get('rho').value
        #print rho, eps_inf
        #N_theta = np.log(threshold)/np.log(rho)
        #print N_theta
        A.semilogx( RMSE[i_image, :], alpha=.7)
        params = mod.make_params(rho=rho)
        A.semilogx(mod.eval(params, x=np.arange(N)), 'r--', alpha=.7)
        A.set_xlabel('l0')
        A.axis('tight')
        A.axis('tight')
        _ = A.set_ylabel('RMSE')            

